# Linear Regression with Pytorch

## Simple Linear Regression Model with sklearn

In [2]:
import pandas as pd
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# 1. load data
cols = ["CRIM",
    "ZN",
    "INDUS",
    "CHAS",
    "NOX",
    "RM",
    "AGE",
    "DIS",
    "RAD",
    "TAX",
    "PTRATIO",
    "B",
    "LSTAT",
    "MEDV",
]
# https://raw.githubusercontent.com/rasbt/python-machine-learning-book/master/code/datasets/housing/housing.data
df = pd.read_csv(
    "../data/raw/housing.data.txt",
    delimiter=r"\s+",
    names=cols,
)
df = df.dropna()
X = df.drop("MEDV", axis=1)
y = df[["MEDV"]]

# 2. Split Train and Test Data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=5
)

# 3. Fit model
reg = linear_model.LinearRegression()
reg.fit(X_train, y_train)

# 4. Predict
y_pred = reg.predict(X_test)

# 4. Measure performane
print(f"MSE: {mean_squared_error(y_test, y_pred)}")

MSE: 20.86929218377065


## Components
### 1. Prediction
### 2. Training

# Prediction
## Mathematical formula
$$
\begin{align}
y=w^Tx+b
\end{align}
$$
- notations
    - **w**: weight vector(D x 1 dim.)
    - **x**: input vector(D x 1 dim.)
    - b: intercept
    - D: the number of features

In [4]:
import numpy as np
D = 10
w = np.ones((D, 1))
x = np.ones((D, 1))
b = 0

# using for loop
prediction = 0
for i in range(D):
    prediction += w[i][0]*x[i][0]
    prediction = prediction+b


# vectorized
prediction = w.T.dot(x)+b

# Training
## Cost function(= Loss function)
### Cost function of linear regression: Mean Squared Error

$$
\begin{align}
J = \frac{1}{N}\sum_{i=1}^N(H(x^{(i)})-y^{(i)})^2
\end{align}
$$
$$
\begin{align}
J = \frac{1}{N}\sum_{i=1}^N((w*x^{(i)}+b)-y^{(i)})^2
\end{align}
$$
 - MAE를 최소가 되게 하는 **w**를 찾는다!


## Optimization Method
- 위에 정의한 cost function을 최대화 하기위해서는 어떠한 과정이 필요한가?
- 선형회귀의 경우, 미분을 사용해서 해를 구할 수 있음!
- 단, 일반적으로는 수리적인 방법(Ex. 동전던지기의 확률을 구한 방법, 미분 등)으로 해를 구할 수 없음
- Optimization Method의 대표적인 예: **Gradient descent**
    - Mathematical Formula
$$
\begin{align}
w \leftarrow w - \eta{\nabla}_wJ
\end{align}
$$
$$
\begin{align}
{\nabla}_nJ=\sum_{n=1}^Nx^{(i)}(H(x^{(i)})-y^{(i)})
\end{align}
$$
<img src="../figures/GradientDescentGIF.gif" alt="drawing" width="600"/>

--------------------

## 조금 더 깊게

- 기계학습의 “학습”은 단순히 모델의 가중치(w)를 찾아내는 것
    - 비유하자면, 새로운 기억이 생성될 때마다, 뇌에 있는 각 시냅스 간의 연결의 세기가 변한다!
- 이러한 관점에서, 기계학습 문제는 단순히 주어진 데이터(X, y)를 가장 잘 설명하는 가중치를 찾아내는 것이다.
- 이러한 가중치를 찾아내는 방법 중 가장 많이 사용되는 것이 최대우도추정(Maximum likelihood Estimation) 방법이다. 

### Base theorem
![basetherom](../figures/baise_theorem.png)

### Likelihood?
<!-- ![likelihoood](../figures/likelihood2.png) -->
<img src="../figures/likelihood2.png" alt="drawing" width="600"/>

In [ ]:
# for i in range()

# gradient_w = X.T.dot(Y-T)

# # 실제 학습 시에는, 업데이트 횟수를 정하거나, loss(cost)의 값의 제한을 둔 후 loop를 돌림
# for epoch in range(num_epochs):
#     w = w - learning_rate*X.T.dot(Y-T)

##

# References
likelihood1: https://jjangjjong.tistory.com/41  
likelihood2: https://angeloyeo.github.io/2020/07/17/MLE.html  
cost function: https://computer-nerd.tistory.com/5  
Deriving Machine Learning Cost Functions using Maximum Likelihood Estimation: https://allenkunle.me/deriving-ml-cost-functions-part1  
Linear Regression Normality: https://stats.stackexchange.com/questions/327427/how-is-y-normally-distributed-in-linear-regression  
gradient descent: https://mccormickml.com/2014/03/04/gradient-descent-derivation/  